<a href="https://colab.research.google.com/github/ecorpnu/portfolio/blob/main/Portfolioassetallocation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

You need to get library ffn, empyrical and PyPortfolioOpt using the pip install. I addded the yfinance as around the time Yahoo was taken over by Verizon, Yahoo decided to change the return data in a format that panda does not recognise, so this is a short work around found in https://github.com/pydata/pandas-datareader/issues/868

In [4]:
pip install ffn


In [5]:
pip install yfinance --upgrade --no-cache-dir

     |████████████████████████████████| 6.3MB 12.1MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.60-py2.py3-none-any.whl size=23819 sha256=2e7878ecabaa5ce330244db85f966b407f82d4ddf15964389e0e9f7b64377ece
  Stored in directory: /tmp/pip-ephem-wheel-cache-ueiha3ol/wheels/f0/be/a4/846f02c5985562250917b0ab7b33fff737c8e6e8cd5209aa3b
Successfully built yfinance
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [6]:
pip install empyrical


     |████████████████████████████████| 61kB 5.0MB/s 
  Created wheel for empyrical: filename=empyrical-0.5.5-cp37-none-any.whl size=39780 sha256=efa03737c382a5851b8df881d744f784182bb5521a10bc2f637414cc0ea02bc2
  Stored in directory: /root/.cache/pip/wheels/ea/b2/c8/6769d8444d2f2e608fae2641833110668d0ffd1abeb2e9f3fc
Successfully built empyrical


In [7]:
pip install PyPortfolioOpt
# see https://pyportfolioopt.readthedocs.io/en/latest/UserGuide.html

     |████████████████████████████████| 61kB 5.3MB/s 
     |████████████████████████████████| 1.3MB 11.5MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for cvxpy: filename=cvxpy-1.1.13-cp37-cp37m-linux_x86_64.whl size=2736156 sha256=d929635c8e769d757c85c3c0fa4520334e9ab389ee3c25863c6e0839fd1cda83
  Stored in directory: /root/.cache/pip/wheels/f9/78/00/f29636789ee83434953b5442f16ec3f9834a68e7fd0393c220
Successfully built cvxpy
  Found existing installation: cvxpy 1.0.31
    Uninstalling cvxpy-1.0.31:
      Successfully uninstalled cvxpy-1.0.31


In [65]:
import ffn 
from empyrical import alpha_beta
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt import discrete_allocation
import matplotlib as pyplot
import numpy as np
import pandas as pd
#the following is needed to work around the problem with yahoo having changed its returned data headings. 
#import pandas 
from pandas_datareader import data as pdr
import yfinance as yfin
yfin.pdr_override()

Here you can play around with the stocks even include ETF. 

In [69]:
prices = ffn.get('IVV,QQQ,BTC-USD,msft,googl,fb',start='2014-09-17')
benchmark = ffn.get('IVV', start='2014-09-17')
qqq = ffn.get('QQQ', start='2014-09-17')
btc = ffn.get('BTC-USD', start='2014-09-17')
msft = ffn.get('msft', start='2014-09-17')
goog = ffn.get('googl', start='2014-09-17')
fb = ffn.get('fb', start='2014-09-17')
ax = prices.rebase().plot()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [46]:
#Problem here with drawing out the graph, need to check out later. 
#For now skip this step
returns = prices.to_returns().dropna()
ax = returns.hist(figsize=(10,10))


In [70]:
stats = prices.calc_stats()
stats.display()

Stat                 ivv         qqq         btcusd      msft        googl       fb
-------------------  ----------  ----------  ----------  ----------  ----------  ----------
Start                2014-09-17  2014-09-17  2014-09-17  2014-09-17  2014-09-17  2014-09-17
End                  2021-07-02  2021-07-02  2021-07-02  2021-07-02  2021-07-02  2021-07-02
Risk-free rate       0.00%       0.00%       0.00%       0.00%       0.00%       0.00%

Total Return         146.53%     283.01%     7311.88%    579.21%     322.25%     364.08%
Daily Sharpe         0.83        1.05        1.23        1.18        0.93        0.88
Daily Sortino        1.25        1.62        2.06        1.96        1.56        1.42
CAGR                 14.21%      21.87%      88.54%      32.60%      23.63%      25.36%
Max Drawdown         -33.90%     -28.56%     -83.04%     -28.04%     -30.87%     -42.96%
Calmar Ratio         0.42        0.77        1.07        1.16        0.77        0.59

MTD                  1.33% 

/usr/local/lib/python3.7/dist-packages/ffn/core.py:2299: RuntimeWarning: divide by zero encountered in true_divide
  res = np.divide(er.mean(), std)


In [71]:
alpha, beta = alpha_beta(btc, benchmark)
print(beta)

155.12999324974712


/usr/local/lib/python3.7/dist-packages/empyrical/stats.py:1296: RuntimeWarning: overflow encountered in power
  out=out


In [75]:
returns = prices.pct_change()
 
# mean daily return and covariance of daily returns
mean_daily_returns = returns.mean()
cov_matrix = returns.cov()
 
# portfolio weights (please make sure weights correspond to your shares above to be 100%, the first refers to the first share)
weights = np.asarray([0.1,0.3,0.1,0.25,0.2,0.05])
 
portfolio_return = round(np.sum(mean_daily_returns * weights) * 252,2)
portfolio_std_dev = round(np.sqrt(np.dot(weights.T,np.dot(cov_matrix, weights))) * np.sqrt(252),2)

print("Expected annualised return: " + str(portfolio_return))
print("Volatility: " + str(portfolio_std_dev))

Expected annualised return: 0.32
Volatility: 0.22


In [76]:
# Expected returns and sample covariance
mu = expected_returns.mean_historical_return(prices)
S = risk_models.sample_cov(prices)

# Optimise portfolio for maximum Sharpe Ratio
ef = EfficientFrontier(mu, S)
raw_weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()
print(cleaned_weights)
#Max Sharpe 
ef.portfolio_performance(verbose=True)

OrderedDict([('ivv', 0.0), ('qqq', 0.0), ('btcusd', 0.2754), ('msft', 0.68394), ('googl', 0.0), ('fb', 0.04066)])
Expected annual return: 47.9%
Annual volatility: 29.6%
Sharpe Ratio: 1.55


(0.4788630646467588, 0.29647687144192264, 1.5477195992222492)

https://github.com/robertmartin8/PyPortfolioOpt/issues/47  (Function updated) . 

total_portfolio_value - you need to input a value here corresponding to how much you are putting into the portfolio in cash so we can calculate the number of shares needed

In [41]:
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices
latest_prices = get_latest_prices(prices)
da = DiscreteAllocation(cleaned_weights, latest_prices, total_portfolio_value=135000)
allocation, leftover = da.lp_portfolio()
print("Discrete allocation number of shares:", allocation)
print("Funds remaining: ${:.2f}".format(leftover))
 

Discrete allocation number of shares: {'btcusd': 1, 'msft': 298, 'googl': 4, 'fb': 3}
Funds remaining: $7278.56


In [42]:
ef = EfficientFrontier(mu, S, weight_bounds=(-1, 1))
ef.efficient_return(target_return=0.2, market_neutral=True)

OrderedDict([('ivv', -0.7159936244535975),
             ('qqq', 0.1160750361526863),
             ('btcusd', 0.1103505945807302),
             ('msft', 0.3898579750206955),
             ('googl', 0.0759716194939204),
             ('fb', 0.0237383992055651)])